In [ ]:
# 因子回测框架
# 首先需要参与回测的股票数据
# 然后确定因子数据
# 通过一个回测框架来因子选股

In [1]:
from data import *

In [2]:
dt = StockData("000300.XSHG", '2010-01-01', '2021-01-01', is_index=True)
dt.print_stock_prices()

close
order_book_id date                
603993.XSHG   2012-10-09    2.6877
              2012-10-10    2.9556
              2012-10-11    2.7017
              2012-10-12    2.5370
              2012-10-15    2.5426
...                            ...
688363.XSHG   2020-12-25  145.0550
              2020-12-28  141.6560
              2020-12-29  133.0407
              2020-12-30  134.9465
              2020-12-31  143.8467

[582409 rows x 1 columns]

In [3]:
# 股票池
list = dt.stock_list
len(list)

300

In [4]:
factor_df = FactorData(list, "market_cap", '2020-01-01', '2021-01-01')
factor_df.print_stock_factors()

market_cap
order_book_id date                    
002230.XSHE   2020-01-02  7.703807e+10
              2020-01-03  7.620261e+10
              2020-01-06  8.073168e+10
              2020-01-07  8.079763e+10
              2020-01-08  7.925863e+10
...                                ...
600570.XSHG   2020-12-25  9.815497e+10
              2020-12-28  9.710044e+10
              2020-12-29  9.833247e+10
              2020-12-30  1.057142e+11
              2020-12-31  1.095251e+11

[72900 rows x 1 columns]

In [5]:
stock_prices = dt.stock_prices
stock_factors = factor_df.factors

In [8]:
# 定义投资周期，比如每个月
rebalance_period = 'M'
# 组合的大小，比如选市值因子最大的前10只股票
portfolio_size = 5
# 获取重整期的日期索引
rebalance_dates = stock_factors.index.get_level_values('date').to_series().resample(rebalance_period).first()
# 保存每个周期投资组合的收益
portfolio_returns = []

In [9]:
# 按周期循环处理
# 循环每一个换仓日
for date in rebalance_dates:
    print("现在的时间是:", date)

    # 在当前周期选取股票
    current_factors = stock_factors.xs(key=date, level='date')

    # 按照市值因子降序排序，取出前N只股票
    selected_stocks = current_factors['market_cap'].nlargest(portfolio_size).index
    print("选出的前5只股票是:", selected_stocks)

    # 获取下一个重新平衡日期，以计算收益
    next_date = rebalance_dates[rebalance_dates > date].min()
    print("下一个换仓日是:", next_date)

    # 计算所选股票从当前日期到下一个日期的收益
    # 如果能找得到下一个换仓日
    if pd.notna(next_date):
        # 找出date与next_date之间的价格数据
        future_prices = stock_prices.loc[(selected_stocks, slice(date, next_date)), 'close'].unstack(level='order_book_id')
        returns = future_prices.pct_change().dropna(how='all').iloc[-1]
        portfolio_returns.append(returns)

        print("选出的股票收益率是:", portfolio_returns)

        # 下一个周期选出来的股票如果在持仓


现在的时间是: 2020-01-02 00:00:00
选出的前10只股票是: Index(['601398.XSHG', '601939.XSHG', '601318.XSHG', '600519.XSHG',
       '601288.XSHG'],
      dtype='object', name='order_book_id')
下一个换仓日是: 2020-02-03 00:00:00
选出的股票收益率是: [order_book_id
601398.XSHG   -0.052456
601939.XSHG   -0.076045
601318.XSHG   -0.069231
600519.XSHG   -0.046429
601288.XSHG   -0.042642
Name: 2020-02-03 00:00:00, dtype: float64]
现在的时间是: 2020-02-03 00:00:00
选出的前10只股票是: Index(['601398.XSHG', '601939.XSHG', '601318.XSHG', '600519.XSHG',
       '601288.XSHG'],
      dtype='object', name='order_book_id')
下一个换仓日是: 2020-03-02 00:00:00
选出的股票收益率是: [order_book_id
601398.XSHG   -0.052456
601939.XSHG   -0.076045
601318.XSHG   -0.069231
600519.XSHG   -0.046429
601288.XSHG   -0.042642
Name: 2020-02-03 00:00:00, dtype: float64, order_book_id
601398.XSHG    0.011333
601939.XSHG    0.013838
601318.XSHG    0.023933
600519.XSHG    0.027446
601288.XSHG    0.014673
Name: 2020-03-02 00:00:00, dtype: float64]
现在的时间是: 2020-03-02 00:00:00
选出的前10只股票是:

In [ ]:
# 回测框架
# 先循环每一个交易日
# 如果到达换仓期，则进行因子选股
# 输出选择做多/做空的股票

In [8]:
import pandas as pd
import numpy as np
from abc import ABC, abstractmethod
from typing import List

class Strategy(ABC):
    """
    抽象策略类，用户需要继承并实现init和next方法。
    """
    def __init__(self, broker, data):
        self.broker = broker
        self.data = data

    @abstractmethod
    def init(self):
        pass
    
    @abstractmethod
    def next(self, bar):
        pass

class Broker:
    """
    用于模拟订单执行和记录交易情况。
    """
    def __init__(self):
        self.orders = []
        self.trades = []

    def buy(self):
        self.orders.append('BUY')

    def sell(self):
        self.orders.append('SELL')

    def execute_orders(self, bar, price):
        if self.orders:
            for order in self.orders:
                self.trades.append({'action': order, 'bar': bar, 'price': price})
            self.orders = []

class Backtest:
    """
    回测引擎，执行回测并输出结果。
    """
    def __init__(self, data: pd.DataFrame, strategy: type[Strategy]):
        self.strategy = strategy
        self.data = data
        self.broker = Broker()

    def run(self):
        strategy = self.strategy(broker=self.broker, data=self.data)
        strategy.init()
        
        for i in range(len(self.data)):
            strategy.next(i)  # 模拟每个时间点的行为
            # 以收盘价作为交易执行价格
            self.broker.execute_orders(bar=i, price=self.data['Close'].iloc[i])
        
        return self.broker.trades

def print_trades(trades):
    for trade in trades:
        print(f"Bar: {trade['bar']}, Action: {trade['action']}, Price: {trade['price']}")



Bar: 19, Action: BUY, Price: 12.240951391428123
Bar: 20, Action: BUY, Price: 98.26986326550849
Bar: 21, Action: BUY, Price: 50.88426450566658
Bar: 22, Action: BUY, Price: 46.51615003504792
Bar: 23, Action: BUY, Price: 2.216833818737496
Bar: 24, Action: BUY, Price: 69.19783760569373
Bar: 25, Action: SELL, Price: 0.03843034439846571
Bar: 26, Action: SELL, Price: 48.516877695056756
Bar: 27, Action: SELL, Price: 23.076879174882826
Bar: 28, Action: SELL, Price: 39.54790696794447
Bar: 29, Action: SELL, Price: 98.10057383694986
Bar: 30, Action: SELL, Price: 9.757423310256675
Bar: 31, Action: SELL, Price: 63.931038614077664
Bar: 32, Action: SELL, Price: 21.577412951148077
Bar: 33, Action: SELL, Price: 68.05638760724455
Bar: 34, Action: SELL, Price: 3.951136379294584
Bar: 35, Action: SELL, Price: 69.52358623318153
Bar: 36, Action: BUY, Price: 72.00204480419862
Bar: 37, Action: BUY, Price: 93.17256807024359
Bar: 38, Action: BUY, Price: 92.96233516105889
Bar: 39, Action: BUY, Price: 18.5211576695

In [10]:
# 示例使用
class SmaCross(Strategy):
    def init(self):
        self.sma1 = self.data['Close'].rolling(10).mean()
        self.sma2 = self.data['Close'].rolling(20).mean()

    def next(self, bar):
        if self.sma1.iloc[bar] > self.sma2.iloc[bar]:
            self.broker.buy()
        elif self.sma1.iloc[bar] < self.sma2.iloc[bar]:
            self.broker.sell()

# 模拟数据
data = pd.DataFrame({'Close': np.random.rand(100) * 100})

bt = Backtest(data, SmaCross)
trades = bt.run()
print_trades(trades)

Bar: 19, Action: BUY, Price: 35.00854723236934
Bar: 20, Action: BUY, Price: 4.074402633702068
Bar: 21, Action: SELL, Price: 15.147318203342708
Bar: 22, Action: SELL, Price: 69.90641638975339
Bar: 23, Action: BUY, Price: 75.3845620986518
Bar: 24, Action: BUY, Price: 59.76490756436985
Bar: 25, Action: SELL, Price: 89.72192031954688
Bar: 26, Action: SELL, Price: 6.776662638297837
Bar: 27, Action: SELL, Price: 53.21635862799127
Bar: 28, Action: SELL, Price: 55.02184992728863
Bar: 29, Action: SELL, Price: 34.9929267701889
Bar: 30, Action: BUY, Price: 85.27059110618028
Bar: 31, Action: BUY, Price: 83.11369928123887
Bar: 32, Action: BUY, Price: 93.07308384455054
Bar: 33, Action: BUY, Price: 79.30937039601082
Bar: 34, Action: BUY, Price: 53.83763946201038
Bar: 35, Action: BUY, Price: 45.149520864720316
Bar: 36, Action: BUY, Price: 59.68764939312028
Bar: 37, Action: BUY, Price: 50.063710794977226
Bar: 38, Action: BUY, Price: 83.42178423083791
Bar: 39, Action: BUY, Price: 74.36657010812642
Bar: 